In [1]:
import traceback
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.2 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=152, type=sensor.camera.rgb), Actor(id=151, type=sensor.other.imu), Actor(id=150, type=sensor.other.gnss), Actor(id=149, type=sensor.other.lane_invasion), Actor(id=148, type=sensor.other.collision), Actor(id=147, type=vehicle.dodge.charger_police_2020), Actor(id=146, type=traffic.traffic_light), Actor(id=145, type=traffic.traffic_light), Actor(id=144, type=traffic.traffic_light), Actor(id=143, type=traffic.traffic_light), Actor(id=142, type=traffic.traffic_light), Actor(id=141, type=traffic.traffic_light), Actor(id=140, type=traffic.traffic_light), Actor(id=139, type=traffic.traffic_light), Actor(id=138, type=traffic.traffic_light), Actor(id=137, type=traffic.traffic_light), Actor(id=136, type=traffic.traffic_light), Actor(id=135, type=traffic.traffic_light), Actor(id=134, type=traffic.traffic_light), Actor(id=133, type=traffic.traffic_light), Actor(id=132, type=traffic.traffic_light), Actor(id=131, type=traffic.traffic_light), Actor(id=130, type=traffic.traffic_light), Actor

In [3]:
ego_vehicle = world.get_actor(147)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=225.252029, y=-364.109375, z=0.000112)


## New Loop 


In [4]:
# initialize sensor data variables
matrix, speed, acceleration, steering_angle, lateral_acceleration, current_yaw, dist_to_lane_center, speed_vehicle_ahead, curvature_degrees_per_meter = None, None, None, None, None, None, None, None, None

previous_speed = 0  # Store the previous speed value in m/s
previous_yaw = 0  # Store the previous yaw angle value
leading_vehicle = None  # Variable to store the leading vehicle reference
previous_time = time.time_ns()

df = initialize_dataframe()
try:
    while True:
        print("----------------------------------")
            
        # update ego location and waypoint
        ego_location = ego_vehicle.get_location()
        ego_waypoint = world_map.get_waypoint(ego_location)
            
        try:
            # create car detection matrix and update parameters
            matrix, params_car_detection = get_car_detection_matrix(**params_car_detection, ego_vehicle=ego_vehicle, ego_waypoint=ego_waypoint, ego_location=ego_location, world=world)
        except Exception as e:
            print("ERROR: In get_car_detection_matrix:")
            print(e)
            print("Traceback:")
            traceback.print_exc()

        current_time = time.time_ns()

        try:
            # get other sensor data
            speed = get_speed(ego_vehicle)
            acceleration = get_acceleration(speed, previous_speed, current_time, previous_time)
            steering_angle = get_steering_angle(ego_vehicle) #TODO angle in degree or radians?
            lateral_acceleration, current_yaw = get_lateral_acceleration(ego_vehicle, speed, previous_yaw, current_time, previous_time)
            dist_to_lane_center = get_dist_to_lane_center(ego_vehicle, world) #in m #TODO needs to be tested with manual car
            speed_vehicle_ahead = get_speed_of_vehicle_ahead(ego_waypoint, world) 

            # Get and print the curvature at the ego vehicle's location in degrees per meter
            curvature_degrees_per_meter = get_curvature_at_location(ego_location, world) 

            # save temporary data
            previous_speed = speed  # Update the previous speed value
            previous_time = current_time
            previous_yaw = current_yaw  # Update the previous yaw angle value
        except Exception as e:
            print("ERROR: In getting sensor data:")
            print(e)
            print("Traceback:")
            print(traceback.print_exc())

        # print sensor data
        print("matrix:")
        if matrix:
            for key, value in matrix.items():
                print(value)
        else:
            print("-- None --")
        print("speed", speed)
        print("acceleration", acceleration)
        print("steering_angle", steering_angle)
        print("lateral_acceleration", lateral_acceleration)
        print("dist_to_lane_center", dist_to_lane_center)
        print("speed_vehicle_ahead", speed_vehicle_ahead)
        print("curvature_degrees_per_meter", curvature_degrees_per_meter)


        # save sensor data in dataframe
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["acceleration"] = acceleration
        row_data["steering_angle"] = steering_angle
        row_data["lateral_acceleration"] = lateral_acceleration
        row_data["dist_to_lane_center"] = dist_to_lane_center
        row_data["speed_vehicle_ahead"] = speed_vehicle_ahead
        row_data["curvature_degrees_per_meter"] = curvature_degrees_per_meter
        df = df._append(row_data, ignore_index=True)

        # clock.tick_busy_loop(60)
        time.sleep(0.3)
        world.tick()
    
finally: 
    display(df)



----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration -0.0
dist_to_lane_center 0.06472260500758349
speed_vehicle_ahead 0.0
curvature_degrees_per_meter -0.0019389209801150733
----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration 0.0
dist_to_lane_center 0.06472260500758349
speed_vehicle_ahead 0.0
curvature_degrees_per_meter -0.0019389209801150733
----------------------------------
Street type: On highway
matrix:
[3, 3, 3, 3, 3, 3, 3, 3]
[

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_0,1_1,...,7_5,7_6,7_7,speed,acceleration,steering_angle,lateral_acceleration,dist_to_lane_center,speed_vehicle_ahead,curvature_degrees_per_meter
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,-0.0,0.064723,0.0,-0.001939
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939
9,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.064723,0.0,-0.001939


KeyboardInterrupt: 

In [ ]:
df.to_csv("test.csv")